In [1]:
import carla
import pandas as pd
import numpy as np
import os
import csv
import sys
import importlib
import json

# Add the "scripts" directory to sys.path
current_dir = os.path.dirname(os.path.abspath("__file__"))
scripts_dir = os.path.abspath(os.path.join(current_dir, 'scripts'))
sys.path.append(scripts_dir)
import LLAMAconnect as llama

### This notebook is used to help find spawnpoints for each place of interest in the map

In [42]:
# Connecting to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(20)
town = 'Town03'
world = client.load_world(f'{town}_Opt') # Town01_Opt is a lighter version of Town01 with layers
world.unload_map_layer(carla.MapLayer.Foliage) # unload foliage for better perfomance 
world.unload_map_layer(carla.MapLayer.Particles) # unload particles for better performance
spectator = world.get_spectator()
spawn_points = world.get_map().get_spawn_points()
interests_spawn_points = {}

In [3]:
def get_closest_spawn_point(transform, spawn_points):
    closest_point = min(spawn_points, key=lambda sp: sp.location.distance(transform.location))
    return closest_point

In [4]:
def draw_all_spawnpoints(spawn_points):
    for i, sp in enumerate(spawn_points):
        world.debug.draw_string(
            sp.location,
            f'O{i}',  # Optional: add index label
            draw_shadow=False,
            color=carla.Color(r=255, g=0, b=0),  # Red
            life_time=500.0,
            persistent_lines=True
     )

In [6]:
def draw_interest_point(interest, nome):

    world.debug.draw_string(
        interest.location,
        f'{nome.upper()}',  # Optional: add index label
        draw_shadow=False,
        color=carla.Color(r=0, g=255, b=0),  # Green
        life_time=500.0,
        persistent_lines=True
    )

In [22]:
def get_spawn_points_from_csv(csv_file):
    spawn_points = {}
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            loc = carla.Location(float(row['x']), float(row['y']), float(row['z']))
            rot = carla.Rotation(float(row['pitch']), float(row['yaw']), float(row['roll']))
            spawn_points[row['Name']] = carla.Transform(loc, rot)
    
    return spawn_points

In [43]:
draw_all_spawnpoints(world.get_map().get_spawn_points())

In [27]:
sps = get_spawn_points_from_csv(f'interests_spawn_points_{town}.csv')
places = list(sps.keys())
places

['home',
 'work',
 'school',
 'gym',
 'bar',
 'theater',
 'cafe',
 'university',
 'restaurant']

In [33]:
draw_interest_point(sps['home'], 'home')

In [54]:
# Run this to create a new interest point in the map
nome = 'restaurant'
interests_spawn_points[nome] = get_closest_spawn_point(spectator.get_transform(), spawn_points)
print(interests_spawn_points[nome])
draw_interest_point(interests_spawn_points[nome], nome)

loc = interests_spawn_points[nome].location
rot = interests_spawn_points[nome].rotation
csv_name = f'interests_spawn_points_{town}.csv'
if os.path.exists(csv_name):
    with open(csv_name, mode='a') as file:
        writer = csv.writer(file)
        writer.writerow([nome, loc.x, loc.y, loc.z, rot.pitch, rot.yaw, rot.roll])
else:
    with open(csv_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'x', 'y', 'z', 'pitch', 'yaw', 'roll'])
        writer.writerow([nome, loc.x, loc.y, loc.z, rot.pitch, rot.yaw, rot.roll])

Transform(Location(x=-145.560196, y=99.700966, z=0.275307), Rotation(pitch=0.000000, yaw=-89.970146, roll=0.000000))


In [55]:
for pl in places:
    draw_interest_point(sps[pl], pl)

In [56]:
importlib.reload(llama)

response = llama.getResponse_trip(places)
while not llama.responseCheck(response, places):
    response = llama.getResponse_trip(places)

response = json.loads(response)
print(response)

{'7': {'location': 'home', 'activity': 'wake up'}, '8': {'location': 'school', 'activity': 'class'}, '9': {'location': 'school', 'activity': 'class'}, '10': {'location': 'school', 'activity': 'class'}, '11': {'location': 'school', 'activity': 'class'}, '12': {'location': 'cafe', 'activity': 'lunch'}, '13': {'location': 'university', 'activity': 'study'}, '14': {'location': 'gym', 'activity': 'sport'}, '15': {'location': 'university', 'activity': 'study'}, '16': {'location': 'university', 'activity': 'study'}, '17': {'location': 'restaurant', 'activity': 'dinner'}, '18': {'location': 'bar', 'activity': 'leisure'}, '19': {'location': 'bar', 'activity': 'leisure'}, '20': {'location': 'theater', 'activity': 'leisure'}, '21': {'location': 'home', 'activity': 'relax'}, '22': {'location': 'home', 'activity': 'relax'}, '23': {'location': 'home', 'activity': 'sleep'}}


In [57]:
old_chdir = os.getcwd()
os.chdir('/opt/carla-simulator/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.basic_agent import BasicAgent
from agents.navigation.behavior_agent import BehaviorAgent
from agents.navigation.custom_agent import CustomAgent
from agents.tools.misc import compute_distance
os.chdir(old_chdir)

In [58]:
def draw_route(routes, life_time=120):
    if not all(isinstance(route, list) for route in routes):
        print("Showing a single route")
        for w in routes:
            world.debug.draw_string(w[0].transform.location, 'O', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=0), life_time=life_time,
            persistent_lines=True)
    else:
        print("Showing multiple routes")
        for route in routes:
            for w in route:
                world.debug.draw_string(w[0].transform.location, 'O', draw_shadow=False,
                color=carla.Color(r=255, g=0, b=0), life_time=life_time,
                persistent_lines=True)

In [59]:
def generate_route(grp, desired_spawn_points, draw_debug_route=True):
    route = []
    for i in range(len(desired_spawn_points) - 1):
        route.append(grp.trace_route(desired_spawn_points[i].location, desired_spawn_points[i + 1].location))
    if draw_debug_route:
        draw_route(route, life_time=99999999)
    return route

In [60]:
amap = world.get_map()
grp = GlobalRoutePlanner(amap, 2)

sps_routine = [] # This is a list of spawnpoint to generate the routine
places_routine = [] # This is a list of places to generate the routine
for locs in response.values():
    local = locs['location']
    places_routine.append(local)
    sps_routine.append(sps[local])

print(places_routine)

generate_route(grp, sps_routine, draw_debug_route=True)

['home', 'school', 'school', 'school', 'school', 'cafe', 'university', 'gym', 'university', 'university', 'restaurant', 'bar', 'bar', 'theater', 'home', 'home', 'home']
Showing multiple routes


[[(<carla.libcarla.Waypoint at 0x7fb924a89200>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a89270>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a892e0>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a89350>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a893c0>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a89430>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a894a0>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a89510>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a89580>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a895f0>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a89660>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924a97040>, <RoadOption.LANEFOLLOW: 4>),
  (<carla.libcarla.Waypoint at 0x7fb924b0c2e0>, <RoadOption.RIGHT: 2>),
  (<